## 余談

ただのポエムです。  

ディープラーニングとハードウェアは切っても切れない関係だと思っている。  
学生の頃から誤差逆伝播法などは既にあったが、NNは計算負荷が高くこんなもの実用的ではない、  
という感じだった。  
当時から問題として上がっていたのは、学習はGPU,FPGAなどでなんとかなるが、  
エッジ側の処理は貧弱なCPUでは厳しいし、電力問題がとても大きいという事だった。  
更にいうとオフライン状況でも使用できる事が望ましいと考えていた。  

当時の研究のターゲットとしては、数mWオーダーで駆動するエッジ側の推論（k-NNを実装していた）専用ASICの開発だった。  
k-NNはとても単純なアルゴリズムで、入力されるデータ（例えば画像データなど）が、  
学習済みでたくさん用意されているデータ（参照データ）のどれに最も近いのかという事をユークリッド距離などの指標を用いて計算し、  
入力と最も近いk個のデータを選ぶというものだ。

これは与えられた入力データと沢山の参照データをSIMD演算で一気に並列で比較する事で高速化できる。  
電力に関してはかなり専門的になるので割愛する。  

専用ASICを作る際には、電力と速度が汎用品(CPUやGPU)を大きく上回る必要がある。  
なぜなら、商用レベルの専用ASICを作ろうと思うと数年の開発期間と数億円〜数十億円の費用がかかってしまい、性能が低いと存在価値がないから。  

ハードウェアで高速化を図る際にはFPGAも有力な選択肢になる。  

## ASICって何？

[ASIC](https://ja.wikipedia.org/wiki/ASIC)は、特定用途のためだけにわざわざLSIとして作るもので、  
CPUのように汎用的に計算に使える類のものではなく、  
例えば、Affine変換をするために内積を計算する必要があるが、  
この内積をAffine変換用に最適化して計算するためだけの[論理回路](https://ja.wikipedia.org/wiki/%E8%AB%96%E7%90%86%E5%9B%9E%E8%B7%AF)を組む。  
Affine変換やCNNなど特定の演算を行う命令セットとして組み込んだCPUを作る場合もある。  
(コンパイラが要るけど)

ASICは上記のように専用の計算をハードウェアで作り込んでしまうために、  
より良い計算アルゴリズムが登場するなどの変化に弱く、  
せっかくハードウェアで作り込んだのに無価値になってしまう可能性がある。  
(Googleほどお金があり、クラウド上での計算効率向上を求めている所であればASICで作り込んだ方が元が取れそう)

## FPGAって何？

[FPGA](https://ja.wikipedia.org/wiki/FPGA)は、ハードウェアをプログラマブルに書き換え可能なデバイスで、  
verilogやVHDLという言語で記述した動作論理をコンパイルして論理回路にする。   
生成されたゲートや接続情報をFPGA内に書き込む事で高速化したい機能をハードウェア化する。  
プログラムを書いて書き込むだけなので開発期間はまあまあ短いし、コストもASICより低い。  

じゃあ、FPGAでいいやん、となりそうだが、  
専用で作り込んだASICと比べると論理回路の電力や速度性能は確実に劣ったものになるしモバイル端末では使えない。  
FPGA自体の単価は高いので、ASICを作った方が開発コストは高いが量産コストは安くつくため、  
沢山売れるのであればASICを作った方が良いかもしれない。  

ASICにせよFPGAにせよ、学習で使う場合にはいくら並列化しても、  
データロードなどの通信やメモリがボトルネックになりそうな気がする。   

## 今後どうなるんだろ

専用のASICを作っている企業は多数あるが、  
ある程度汎用的に使えるASICとなるとアプリケーションとの親和性も重要になってきそう。  

DPU面白い、低電力化のために駆動率を下げるという意味でもデータフローは良い気がする。  
GALSかぁ、ちゃんと読んでないけど同期回路に比べてFIFOお化けになりそう・・だが、  
動的に電源電圧を制御しているのか分からないが電力面では有利だろうし、  
クロックノイズの影響も低減できるし、製造ばらつきの影響を平均化してくれるので、  
DPUはかなり電力を考慮した作りになっている気がする。  
時間ある時に真面目に読んでみたい。  
https://news.mynavi.jp/article/20170502-dpu/  
https://news.mynavi.jp/article/20171018-hotchips29_dpu/

Mythicはアナログ回路を使って積和演算を行なっているっぽい。  
アナログ回路的にニューロンを実現する研究は昔されていた気がする、ニューロンMOSFETとかISSCCで発表されていたな。  
https://tohoku.repo.nii.ac.jp/?action=repository_action_common_download&item_id=53578&item_no=1&attribute_id=18&file_no=1
https://news.mynavi.jp/article/20180912-691798/

アプリケーションとの親和性が高く、電力を意識した設計ノウハウを持つところが勝ちそうな気がする。スヤァ  